In [18]:
import gym
import matplotlib.pyplot as plt
import numpy as np
from misc_tools.print_latex import print_tex

In [19]:
env=gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True,render_mode="ansi")#,render_mode="human"
env.reset()
#env.close()
def p_sp_sa(state, action):
    1
print(env.render())
rewards = np.array(env.reward_range)# [0,1]
num_states= env.observation_space.n
num_actions = env.action_space.n



SFFF
FHFH
FFFH
HFFG



In [20]:
gamma = 0.9
policy = np.ones((num_states, num_actions))/num_actions
state_value = np.zeros((num_states,))
p_spr_sa = np.zeros((num_states,  num_actions, num_states, rewards.size))
for state, actions_d in env.P.items():
    for action, transitions in actions_d.items():
        for (p, state_tar, reward, is_terminal) in transitions:
            p_spr_sa[state, action, state_tar, int(reward)] += p   
            # apparently walls actions transition to same state = different actions to same state. so += p
print(f'(s\', a) normalized: {np.all(p_spr_sa.sum(axis=(-2,-1)) == 1)}')
#norm = p_spr_sa.sum(axis=(-2,-1), keepdims = 1)
#p_spr_sa = np.divide(p_spr_sa, norm, out=np.zeros_like(p_spr_sa), where= (norm != 0))
p_spr_sa[14,1].sum(-1)

(s', a) normalized: True


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.33333333, 0.33333333,
       0.33333333])

from [RL_Reinforced_Learning/Meaning_of_Psasr_probability_matrix.ipynb](data_processing/neural_networks/RL_Reinforced_Learning/Meaning_of_Psasr_probability_matrix.ipynb)

$p(s^\prime, r|s,a)$ is $P[s,a]$, an entry of matrix $P$; $r_i$ is i-th entry $ \vec{R}[i]$ of vector $ \vec{R}$.
$$\sum_{r \in R} r \cdot p(s^\prime, r|s,a)$$
is a $[s,a]$ entry of matrix multiplication 
$$Q[s,a] = P[s,a]  \vec{R}; \ \text{dim}(Q) = [s^\prime,r]\times[r] = [s^\prime]$$
$$\boxed{r(s,a) =  \sum_{r \in R} \sum_{s^\prime \in S} r \cdot p(s^\prime, r|s,a) = \sum_{s^\prime \in S} (P \vec{R})_{s,a,s^\prime}}$$
***



$$\boxed{r(s,a,s^\prime) = \frac{\sum_r  r\cdot p(s^\prime,r|s,a)}{p(s^\prime|s,a)}}$$
$$\boxed{
p(s^\prime|s,a) = \sum_r p(s^\prime,r|s,a)}$$
$$q_{\pi}(s,a) = \sum_{r \in R} \sum_{s^\prime \in S} p(s^\prime, r|s,a) \cdot \biggl[ r + \gamma \cdot v_{\pi}(s^\prime)\biggr]=$$
$$ =  \sum_{s^\prime \in S} \left[\sum_{r \in R} p(s^\prime, r|s,a) \cdot r + \gamma \sum_{r \in R} p(s^\prime, r|s,a) \cdot v_{\pi}(s^\prime)\right] = $$
$$ = \sum_{s^\prime \in S} \left[p(s^\prime|s,a)\cdot r(s,a,s^\prime) + \gamma \cdot v_{\pi}(s^\prime) \cdot \sum_{r \in R} p(s^\prime, r|s,a) \right] =$$ 
$$ = \sum_{s^\prime \in S} \biggl[  p(s^\prime|s,a)\cdot r(s,a,s^\prime) + \gamma \cdot v_{\pi}(s^\prime) \cdot p(s^\prime|s,a) \biggr] =$$ 
$$= \sum_{s^\prime \in S} p(s^\prime|s,a)\cdot\biggl[r(s,a,s^\prime) + \gamma \cdot v_{\pi}(s^\prime)\biggr]


In [21]:
p_sp_sa = p_spr_sa.sum(-1)
r_sas_top = p_spr_sa @ rewards
r_sas = np.divide(r_sas_top, p_sp_sa, out = np.zeros_like(r_sas_top), where= (p_sp_sa != 0))

#a = r_sas + gamma*state_value
#print_tex((p_sp_sa* a).sum((-1)))
#r_sas[14,1,15], p_sp_sa[14,1,15]
#r_sas_top.shape, p_sp_sa.shape, r_sas.shape

***
$$v_{\pi}(s) = \sum_{a \in A} \pi(a|s) \sum_{r \in R} \sum_{s^\prime \in S} p(s^\prime, r|s,a) [ r + \gamma v_{\pi}(s^\prime)] = $$
$$ = \sum_{a \in A} \pi(a|s) \cdot q_{\pi}(s,a) $$

In [28]:
#PR = (p_spr_sa @ rewards)#.sum(axis=(-1))

def get_q_sa(state_value):
   return (p_sp_sa * (r_sas + gamma*state_value)).sum(-1)

def evaluate_policy(state_action_value, policy):
   v_pi = (policy * state_action_value).sum(-1)
   return v_pi

def evaluate_policy_k(state_value, policy, k):
   v_pi = state_value.copy()
   for _ in range(k):
      q_pi  = get_q_sa(v_pi)
      v_pi  = evaluate_policy(q_pi, policy)
   q_pi  = get_q_sa(v_pi)
   return v_pi, q_pi

def improve_policy(state_action_value):
   a = state_action_value
   max_values = np.max(a, axis=-1)
   max_indices = [np.where(a[i] == max_values[i])[0] for i in range(a.shape[0])]
   max_indices

   one_hot_matrix = np.zeros_like(a)

   for i, indices in enumerate(max_indices):
      one_hot_matrix[i, indices] = 1/len(indices)
   max_indices,one_hot_matrix

   return one_hot_matrix

v_pi  = state_value.copy()
pi    = policy.copy()
for _ in range(50):
   v_pi, q_pi = evaluate_policy_k(v_pi, pi, 50)
   pi = improve_policy(q_pi)
   

#print_tex(v_pi.reshape(4,4))
print_tex(v_pi.reshape(4,4),pi,q_pi)

#print_tex(sv2.reshape(4,4),valueFunctionVectorComputed.reshape(4,4))
# print(env.render())


<IPython.core.display.Math object>

In [23]:
def evaluatePolicy(env,valueFunctionVector,policy,discountRate,maxNumberOfIterations,convergenceTolerance):
    import numpy as np
    convergenceTrack=[]
    for iterations in range(maxNumberOfIterations):
        convergenceTrack.append(np.linalg.norm(valueFunctionVector,2))
        valueFunctionVectorNextIteration=np.zeros(env.observation_space.n)
        for state in env.P:
            outerSum=0
            for action in env.P[state]:
                innerSum=0
                for probability, nextState, reward, isTerminalState in env.P[state][action]:
                    #print(probability, nextState, reward, isTerminalState)
                    innerSum=innerSum+ probability*(reward+discountRate*valueFunctionVector[nextState])
                outerSum=outerSum+policy[state,action]*innerSum
            valueFunctionVectorNextIteration[state]=outerSum
        if(np.max(np.abs(valueFunctionVectorNextIteration-valueFunctionVector))<convergenceTolerance):
            valueFunctionVector=valueFunctionVectorNextIteration
            print('Iterative policy evaluation algorithm converged!')
            break
        valueFunctionVector=valueFunctionVectorNextIteration       
    return valueFunctionVector

def improvePolicy(env,valueFunctionVector,numberActions,numberStates,discountRate):
    import numpy as np
    # this matrix will store the q-values (action value functions) for every state
    # this matrix is returned by the function 
    qvaluesMatrix=np.zeros((numberStates,numberActions))
    # this is the improved policy
    # this matrix is returned by the function
    improvedPolicy=np.zeros((numberStates,numberActions))
     
    for stateIndex in range(numberStates):
        # computes a row of the qvaluesMatrix[stateIndex,:] for fixed stateIndex, 
        # this loop iterates over the actions
        for actionIndex in range(numberActions):
            # computes the Bellman equation for the action value function
            for probability, nextState, reward, isTerminalState in env.P[stateIndex][actionIndex]:
                qvaluesMatrix[stateIndex,actionIndex]=qvaluesMatrix[stateIndex,actionIndex]+probability*(reward+discountRate*valueFunctionVector[nextState])
             
        # find the action indices that produce the highest values of action value functions
        bestActionIndex=np.where(qvaluesMatrix[stateIndex,:]==np.max(qvaluesMatrix[stateIndex,:]))
 
        # form the improved policy        
        improvedPolicy[stateIndex,bestActionIndex]=1/np.size(bestActionIndex)
    return improvedPolicy,qvaluesMatrix


In [24]:
discountRate=0.9
stateNumber=16
actionNumber=4

maxNumberOfIterationsOfPolicyIteration=30
initialPolicy=(1/actionNumber)*np.ones((stateNumber,actionNumber))
valueFunctionVectorInitial=np.zeros(env.observation_space.n)
# maximum number of iterations of the iterative policy evaluation algorithm
maxNumberOfIterationsOfIterativePolicyEvaluation=1000
# convergence tolerance 
convergenceToleranceIterativePolicyEvaluation=10**(-6)
currentPolicy=initialPolicy
valueFunctionVectorComputed =evaluatePolicy(env,valueFunctionVectorInitial,currentPolicy,discountRate,maxNumberOfIterationsOfIterativePolicyEvaluation,convergenceToleranceIterativePolicyEvaluation)
improvedPolicy,qvaluesMatrix=improvePolicy(env,valueFunctionVectorComputed,actionNumber,stateNumber,discountRate)
valueFunctionVectorComputed.reshape(4,4)
#improvedPolicy
# qvaluesMatrix


Iterative policy evaluation algorithm converged!


array([[0.00447465, 0.00422061, 0.01006512, 0.00411696],
       [0.00672042, 0.        , 0.02633306, 0.        ],
       [0.01867524, 0.05760645, 0.10697145, 0.        ],
       [0.        , 0.13038262, 0.39148975, 0.        ]])